In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import torch

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

import utils.torchUtils as gnn

In [2]:
template = gnn.Dataset("data/template", make_template=True)

In [3]:
model = gnn.GoldenGCN.load_from_checkpoint(
    "/uscms_data/d3/ekoenig/8BAnalysis/studies/sixbStudies/jupyter/eightb/pairing_methods/graph_net/models/golden_classifier/lightning_logs/version_2/checkpoints/epoch=24-step=9024.ckpt",
    dataset=template)

In [4]:
import utils.torchUtils.cpp_geometric as cpg 

cpg.export_model(model,template,'cpg_models/golden_gcn/')

Exporting model to: cpg_models/golden_gcn/
[model]
num_layers = 10
layers = GCNConvMSG,GCNRelu,GCNConvMSG,GCNRelu,NodeLinear,EdgeLinear,GCNRelu,NodeLinear,EdgeLinear,GCNLogSoftmax
layer_shapes = 32,14,0,0,128,160,0,0,64,128,64,384,0,0,2,64,2,64,0,0

[features]
num_node_features = 5
node_features = m,pt,eta,phi,btag
num_node_mask = 5
node_mask = m,pt,eta,phi,btag
num_edge_features = 4
edge_features = dpt,dr,deta,dphi
num_edge_mask = 4
edge_mask = dpt,dr,deta,dphi
scale = standardize

[scaler]
node_scale_min = 0.62060857,20.0,-2.4995117,-3.1415837,0.0010824203
node_scale_max = 432.17313,3893.0654,2.4995117,3.1415837,0.9995117
node_scale_mean = 13.717373076359248,98.03337467019047,-0.002745888030241829,-4.481559994330364e-06,0.5525142590277192
node_scale_std = 10.58819136574055,91.36370297721268,1.0554675782990282,1.816123314169241,0.42406181873769
edge_scale_min = -3866.545654296875,0.0,-4.99755859375,-3.1415913740741175
edge_scale_max = 3866.545654296875,5.852164781833368,4.99755859375,

In [5]:
ndata = 1000

In [6]:
template = gnn.Dataset("data/template", make_template=True, scale='raw')
dataset = gnn.Dataset("data/MX_1200_MY_500-testing", transform=template.transform)[:ndata]

In [7]:
cpg.export_graphs(dataset, outdir='cpg_test_data/golden_csv/')

In [8]:
template = gnn.Dataset("data/template", make_template=True, scale='standardize')
dataset = gnn.Dataset("data/MX_1200_MY_500-testing", transform=template.transform)[:ndata]

In [9]:
cpg.export_graphs(dataset, outdir='cpg_test_data/golden_csv/', tag='scaled')

In [10]:
node_os, edge_os = [], []
with torch.no_grad():
    for data in dataset:
        node_o, edge_o = model(data)
        node_os.append(node_o.numpy())
        edge_os.append(edge_o.numpy())

In [11]:
cpg.export_nodes(node_os, outdir='cpg_test_data/golden_csv/', tag='golden')
cpg.export_edges(edge_os, outdir='cpg_test_data/golden_csv/', tag='golden')
